Для выполнения задачи напишем Bash-скрипт и создадим systemd-юнит для его автоматического запуска при старте системы. Скрипт будет проверять процесс test, отправлять запросы на сервер мониторинга и вести логирование.

In [ ]:
1. Bash-скрипт
Создадим файл /usr/local/bin/monitor_test.sh с содержимым:

In [ ]:
#!/bin/bash

# Лог-файл
LOG_FILE="/var/log/monitoring.log"

# URL для мониторинга
MONITORING_URL="https://test.com/monitoring/test/api"

# Имя процесса
PROCESS_NAME="test"

# Функция для записи в лог
log_message() {
    echo "$(date '+%Y-%m-%d %H:%M:%S') $1" >> "$LOG_FILE"
}

# Проверяем, запущен ли процесс
if pgrep -x "$PROCESS_NAME" > /dev/null; then
    # Если процесс запущен, отправляем запрос на сервер мониторинга
    if ! curl -s -o /dev/null -w "%{http_code}" "$MONITORING_URL" | grep -q "200"; then
        log_message "Сервер мониторинга недоступен"
    fi

    # Проверяем, был ли процесс перезапущен
    if [ ! -f "/tmp/${PROCESS_NAME}_pid" ]; then
        # Сохраняем PID процесса
        pgrep -x "$PROCESS_NAME" > "/tmp/${PROCESS_NAME}_pid"
    else
        # Сравниваем текущий PID с сохраненным
        SAVED_PID=$(cat "/tmp/${PROCESS_NAME}_pid")
        CURRENT_PID=$(pgrep -x "$PROCESS_NAME")
        if [ "$SAVED_PID" != "$CURRENT_PID" ]; then
            log_message "Процесс $PROCESS_NAME был перезапущен"
            echo "$CURRENT_PID" > "/tmp/${PROCESS_NAME}_pid"
        fi
    fi
fi

Не забудьте сделать скрипт исполняемым:

In [ ]:
sudo chmod +x /usr/local/bin/monitor_test.sh

Настройка systemd:
1. Создание unit-файла systemd:
Создайте файл /etc/systemd/system/monitor_test.service с следующим содержимым:

In [ ]:
[Unit]
Description=Monitor Test Process
After=network.target

[Service]
ExecStart=/usr/local/bin/monitor_test.sh
Type=oneshot

[Install]
WantedBy=multi-user.target

2. Создание таймера systemd:
Создайте файл /etc/systemd/system/monitor_test.timer с следующим содержимым:

In [ ]:
[Unit]
Description=Run Monitor Test Process every minute

[Timer]
OnBootSec=1min
OnUnitActiveSec=1min
Unit=monitor_test.service

[Install]
WantedBy=timers.target

3. Активирование и запуск таймера:

In [ ]:
sudo systemctl daemon-reload
sudo systemctl enable monitor_test.timer
sudo systemctl start monitor_test.timer

Проверка работы

Убедимся что таймер активен:

In [ ]:
systemctl list-timers | grep monitor_test

Проверим лог-файл /var/log/monitoring.log для подтверждения работы скрипта:

In [ ]:
tail -f /var/log/monitoring.log